In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from iminuit import cost, Minuit

In [2]:
data = pd.read_excel("a0980def2.xlsx", sheet_name = "Dataset")
M_eta_pi = data['Mass'] # M
events = data['Event']
error = 0.5*(data['hErr'] - data['lErr'])

Ma0980 = 0.9825
M_eta = 0.548
M_pi = 0.14
M_K = 0.493
M_etaC = 2.984
GR = 0.112

In [3]:

def k_i(channel, M):
    if channel == 1:
        return np.sqrt(M**2 - (M_pi + M_eta)**2)
    elif channel == 2:
        return np.sqrt(M**2 - (2 * M_K)**2 +0j)
    
def lbd(x, y, z):
    return x**2+y**2+z**2-2*(x*y + x*z + y*z)

def phaseSpace(M):
    def p(M):
        return np.sqrt(lbd(M**2, M_etaC**2, M_pi**2))/(2*M_etaC)
    def q(M):
        return np.sqrt(lbd(M**2, M_eta**2, M_pi**2))/(2*M)
    return p(M)*q(M)*M_etaC

def productionTerm(M, p0, p1):
    return p0 + p1*M**2

def backgroundTerm(M, b0, b1):
    return b0 + b1*M**2

def T11(M, m11, m22, m12_squared):
    return (m22 - 1j*k_i(2, M))/((m11 - 1j*k_i(1, M))*(m22 - 1j*k_i(2, M)) - m12_squared)

def intensity(M, p0, p1, m11, m22, m12_squared, b0, b1):
    return phaseSpace(M)*(np.power(abs(productionTerm(M, p0, p1)*T11(M, m11, m22, m12_squared)),2) + backgroundTerm(M, b0, b1))


In [4]:
costFun = cost.LeastSquares(M_eta_pi, events, error, intensity)
m1 = Minuit(costFun, p0 = 0.3, p1 = 0.01, m11 = 0.1, m22 = 0., m12_squared = 0.28998, b0 = 0., b1 = -0.5)
m1.migrad()
m1.minos()


┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 18.22 (χ²/ndof = 0.9)      │             Nfcn = 2733              │
│ EDM = 9.26e-06 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬─────────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name        │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼─────────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ p0          │    30     │     4     │     -4     │     4      │         │         │       │
│ 1 │ p1          │   -19.4   │    3.2    │    -3.3    │    3.3     │         │         │       │
│ 2 │ m11         │   -0.50   │   0.19    │   -0.23    │    0.18    │         │         │       │
│ 3 │ m22         │  -0.578   │   0.023   │   -0.032   │   0.019    │         │         │       │
│ 4 │ m12_squared │   0.47    │   0.11    │   -0.09    │    0.16    │         │         │       │
│ 5 │ b0          │    46     │     9     │     -9     │     9      │         │         │       │
│ 6 │ b1          │    22     │     7     │     -6     │     7      │         │         │       │
└───┴─────────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌──────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┐
│          │          p0           │          p1           │          m11          │          m22          │      m12_squared      │          b0           │          b1           │
├──────────┼───────────┬───────────┼───────────┬───────────┼───────────┬───────────┼───────────┬───────────┼───────────┬───────────┼───────────┬───────────┼───────────┬───────────┤
│  Error   │    -4     │     4     │   -3.3    │    3.3    │   -0.23   │   0.18    │  -0.032   │   0.019   │   -0.09   │   0.16    │    -9     │     9     │    -6     │     7     │
│  Valid   │   True    │   True    │   True    │   True    │   True    │   True    │   True    │   True    │   True    │   True    │   True    │   True    │   True    │   True    │
│ At Limit │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │
│ Max FCN  │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │
│ New Min  │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │   False   │
└──────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────────┴───────────┘
┌─────────────┬─────────────────────────────────────────────────────────────────────────────────────┐
│             │          p0          p1         m11         m22 m12_squared          b0          b1 │
├─────────────┼─────────────────────────────────────────────────────────────────────────────────────┤
│          p0 │        15.3         -12       -0.06    -21.8e-3       0.233